 # Mean Reciprocal Rank (MRR)

In [34]:
import pandas as pd
import seaborn as sns

palette = sns.color_palette(["#2176AB", "#F97662", "#FFBF00", "#50C878", "#B284BE"])
sns.set_palette(palette)


llm_res = pd.read_csv('../gym/results/llm_response.csv')
altair_res = pd.read_csv('../gym/results/altair_response.csv')

llm_res.drop(columns=['model', 'index', 'file_id', 'table_id'], inplace=True)
llm_res = llm_res.groupby(['id', 'query_id']).mean().reset_index()

altair_res = altair_res[(altair_res['threshold']*10).astype(int) == 3]
altair_res.drop(columns=['query', 'threshold', 'file_id', 'table_id'], inplace=True)

# get a df with all
df = pd.merge(llm_res, altair_res, on=['id', 'query_id'], how='inner')
df

,id,query_id,score,relevant,order
0,1609.05600v2_S5.T1,1,0.075000,0.00,0
1,1609.05600v2_S5.T1,2,0.050000,0.00,108
2,1609.05600v2_S5.T1,4,0.250000,0.25,1
3,1609.05600v2_S5.T1,5,0.025000,0.00,16
4,1609.05600v2_S5.T1,9,0.837500,1.00,1
...,...,...,...,...,...
558,2410.13042v1_Sx1.T0,10,0.050000,0.00,25
559,2410.13042v1_Sx1.T0a,2,0.025000,0.00,134
560,2410.13042v1_Sx1.T0a,5,0.175000,0.25,23
561,2410.13042v1_Sx1.T0a,6,0.442500,0.50,12


In [35]:
# kendal correlation between df order and score
from scipy.stats import spearmanr

print(spearmanr(df['relevant'], df['score']))
print(spearmanr(df['order'], df['score']))


SignificanceResult(statistic=0.5951268688933677, pvalue=3.0781480642802495e-55)
SignificanceResult(statistic=-0.26236122418374264, pvalue=2.5743793715020916e-10)


In [36]:
scores = []

for i in range(1, 11):
    altair_table = altair_res[altair_res['query_id'] == i].sort_values('order')
    llm_table = llm_res[(llm_res['query_id'] == i) & (llm_res['relevant'] >= 0.2)]

    # get the first relevant document in altair_table
    # through the altair table to get the first relevant document in the llm table
    for j in range(len(altair_table)):
        if altair_table.iloc[j]['id'] in llm_table['id'].values:
            scores.append(1 / (j + 1))
            break

mrr = sum(scores) / len(scores)
print(f"MRR: {mrr}")

MRR: 0.6207912457912458
